In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import openpyxl  # openpyxl 모듈 추가
import re

sns.set_style('darkgrid')
sns.set_palette('Set3')
warnings.filterwarnings('ignore')
plt.rcParams["font.family"] = "NanumBarunGothic"


In [2]:
from datetime import datetime, timedelta

headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# 시작일과 종료일 설정
start_day = datetime(2024, 3, 1)
end_day = datetime(2024, 4, 29)

# 시작일과 종료일 사이의 주count 계산
total_weeks = ((end_day - start_day).days // 7) + 1

# 엑셀 파일 생성
wb = openpyxl.Workbook() 
ws = wb.active
ws.append(["Day", "SongId", "Rank", "Title_Name", "Singer","ImgURL"])

# 주 단위로 날짜 출력
current_day = start_day
print(f"시작일 : {start_day}, 종료일 : {end_day}")
Week_Count = 0
   
while current_day <= end_day:
   # 주의 시작일과 끝일을 계산
   week_start = current_day.strftime("%Y%m%d")
   week_end = (current_day + timedelta(days=6)).strftime("%Y%m%d")
   # print(f"{week_start}~{week_end}")
   current_day += timedelta(weeks=1)
   
   date =  (current_day + timedelta(days=0)).strftime("%Y%m%d")
   # URL 뒤에 들어갈 날짜
   print(date)
   
   # pg=1 1~50등 pg=2 51~100등
   response = requests.get("https://www.genie.co.kr/chart/top200?ditc=W&ymd={0}&hh=12&rtm=N&pg=1".format(date), headers=headers)
   
   URL = "https://www.genie.co.kr/chart/top200?ditc=W&ymd={0}&hh=12&rtm=N&pg=1".format(date)
   
   print(URL)
   
   html = response.content.decode('utf-8','replace')
   soup = BeautifulSoup(html, 'html.parser',from_encoding='cp949')

   my_st = soup.select(
      '#body-content > div.newest-list > div > table > tbody'
   )
   
   
   for i in my_st:
      for Rank_Count in range(1,51):
        # 뮤직 ID
        # 모든 <tr> 태그 찾기
        trs = i.select('tr:nth-child({0})'.format(Rank_Count))

        # 각 <tr> 태그에서 'songid' 속성 값 추출
        for tr in trs:
            if 'songid' in tr.attrs:
                songid_value = tr['songid']
        # Rank 정보를 추출하는 부분
        rank = i.select("tr:nth-child({0}) > td.number".format(Rank_Count))
        # 정규 표현식을 사용하여 숫자만 추출
        Rank_num = [int(re.search(r'\d+', r.text).group()) for r in rank]

        # 곡 제목 추출
        title = i.select("tr:nth-child({0}) > td.info > a.title.ellipsis".format(Rank_Count))
        Title_text = [t.text.strip() for t in title]

        # 아티스트 추출
        artist = i.select("tr:nth-child({0}) > td.info > a.artist.ellipsis".format(Rank_Count))
        Artist_text = [a.text.strip() for a in artist]

        # 커버 사진 링크 추출
        Cover_img = [img['src'] for img in i.select("tr:nth-child({0}) > td:nth-child(3) > a > img".format(Rank_Count))]

        for Rank_num, Title_text, Artist_text, Cover_img in zip(Rank_num, Title_text, Artist_text, Cover_img):
                 ws.append([date, songid_value, Rank_num, Title_text, Artist_text, Cover_img])

# 엑셀 파일 저장
file_path = r"C:\P_Project\1.Project\5-Project_data\2.Genie_No_Detail4.29.2.xlsx"
wb.save(file_path)
print("Excel 파일이 저장되었습니다:", file_path)


시작일 : 2024-03-01 00:00:00, 종료일 : 2024-04-29 00:00:00
20240308
https://www.genie.co.kr/chart/top200?ditc=W&ymd=20240308&hh=12&rtm=N&pg=1
20240315
https://www.genie.co.kr/chart/top200?ditc=W&ymd=20240315&hh=12&rtm=N&pg=1
20240322
https://www.genie.co.kr/chart/top200?ditc=W&ymd=20240322&hh=12&rtm=N&pg=1
20240329
https://www.genie.co.kr/chart/top200?ditc=W&ymd=20240329&hh=12&rtm=N&pg=1
20240405
https://www.genie.co.kr/chart/top200?ditc=W&ymd=20240405&hh=12&rtm=N&pg=1
20240412
https://www.genie.co.kr/chart/top200?ditc=W&ymd=20240412&hh=12&rtm=N&pg=1
20240419
https://www.genie.co.kr/chart/top200?ditc=W&ymd=20240419&hh=12&rtm=N&pg=1
20240426
https://www.genie.co.kr/chart/top200?ditc=W&ymd=20240426&hh=12&rtm=N&pg=1
20240503
https://www.genie.co.kr/chart/top200?ditc=W&ymd=20240503&hh=12&rtm=N&pg=1
Excel 파일이 저장되었습니다: C:\P_Project\1.Project\5-Project_data\2.Genie_No_Detail4.29.2.xlsx
